In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

Import the data

In [ ]:
forecast_df = pd.read_excel('data.xlsx', sheet_name=0)
meters_df = pd.read_excel('data.xlsx', sheet_name=1)
rates_df = pd.read_excel('data.xlsx', sheet_name=2)

display(forecast_df)
display(meters_df)
display(rates_df)

# 1. Calculate transportation cost for the provided dataset

In [ ]:
def calc_transportation_cost(forecast_df, meters_df, rates_df):
    merged_df = pd.merge(meters_df, rates_df, how='right', on='Exit Zone').dropna(subset=['Meter ID'])
    
    # Select only the relevant rate from imported rates_df based on Annual Quantity
    merged_df['Annual Quantity (Max)'] = merged_df['Annual Quantity (Max)'].fillna(np.inf)
    merged_df = merged_df[(merged_df['Annual Quantity (kWh)'] >= merged_df['Annual Quantity (Min)']) & (merged_df['Annual Quantity (kWh)'] < merged_df['Annual Quantity (Max)'])]
    merged_df.sort_values(['Date'], inplace=True)
    
    # Create new df to store results
    df = pd.DataFrame(columns=forecast_df.columns)
    
    for grp, grp_df in merged_df.groupby('Meter ID'):
        # Generate series to map to correct rate in forecast data
        series = pd.Series(
            grp_df['Rate (p/kWh)'].tolist()+[np.nan], pd.Index(grp_df['Date'].tolist()+[pd.Timestamp('2024-09-30')])
        ).asfreq('D').fillna(method='ffill')
        tmp_df = forecast_df[forecast_df['Meter ID'] == grp].copy()
        tmp_df['rate'] = forecast_df['Date'].map(series)
        
        df = df.append(tmp_df)
    
    df['Cost (p)'] = df['kWh'] * df['rate']
    df['Cost (£)'] = df['Cost (p)'] / 100

    result = df[['Meter ID', 'kWh', 'Cost (£)']].copy()
    result = result.groupby('Meter ID').sum().rename(columns={
        'Cost (£)': 'Total Cost (£)',
        'kWh': 'Total Estimated Consumption (kWh)'
    }).reset_index().round(2)
    
    return result

cost_df = calc_transportation_cost(forecast_df, meters_df, rates_df)

cost_df

# 2. Generate random meters with random AQ using Exit Zones from the provided rates table

In [ ]:
def make_meters(rates_df, number):
    
    ids = []
    while len(ids) < number:
        random_id = np.random.randint(10000000, 1000000000)
        if random_id not in ids:
            ids.append(random_id)
            
    valid_zones = np.random.choice(rates_df['Exit Zone'].unique().tolist(), number)
            
    aq = [np.random.randint(0, 1000000) for _ in ids]
    
    return pd.DataFrame({'Meter ID': ids, 'Exit Zone': valid_zones, 'Annual Quantity (kWh)': aq})

# Specify the number of meters to generate
new_meters_df = make_meters(rates_df, 20)

new_meters_df

# 3. Generate mock consumption from a list of meters and a time period

In [ ]:
def make_mock_consumption(meters, start, end):
    df = pd.DataFrame(columns=['Date', 'Meter ID', 'kWh'])
    for meter in meters:

        dates = pd.Index([pd.Timestamp(start), pd.Timestamp(end)])
        series = pd.Series([meter]*2, dates).asfreq('D').fillna(meter).apply(int)
        tmp_df = pd.DataFrame(series, columns=['Meter ID'])
        
        tmp_df['Date'] = tmp_df.index
        
        consumption = np.random.choice(np.arange(1, 999), len(tmp_df))
        noise = np.random.normal(0, 1, consumption.shape)
        consumption = consumption + noise
        tmp_df['kWh'] = consumption
        
        df = df.append(
            tmp_df,
            ignore_index=True
        )
        
    return df
    
random_meters = np.random.choice(new_meters_df['Meter ID'], np.random.randint(1, len(new_meters_df)))
start = '2018-04-01'
end = '2025-10-01'

random_forecast_df = make_mock_consumption(random_meters, start=start, end=end)

random_forecast_df

# 4. I already wrote a function in '1' that will generate the transportation cost table. Instead, just write a simple benchmark here.

In [ ]:
def benchmark(rates_df, number_of_meters, number_of_years):
    
    # Make the benchmark df to store results
    benchmark_df = pd.DataFrame(columns=['number_of_meters', 'number_of_years', 'execution_time'])
    benchmark_df['number_of_meters'] = number_of_meters * len(number_of_years)
    benchmark_df['number_of_years'] = [i for i in number_of_years for ii in range(len(number_of_meters))]
    
    for index, row in benchmark_df.iterrows():
        # Generate random forecast data with each iteration   
        meters_df = make_meters(rates_df, row['number_of_meters'])
        forecast_df = make_mock_consumption(
            meters_df['Meter ID'].unique().tolist(),
            pd.Timestamp('2020-04-01'), pd.Timestamp('2020-04-01') + pd.Timedelta(days=round(365*row['number_of_years']))
        )

        # Calculate transport cost and capture execution time
        start_t = time.time()
        df = calc_transportation_cost(forecast_df, meters_df, rates_df)
        total_t = time.time() - start_t
        
        benchmark_df.loc[index, 'execution_time'] = total_t
        
        print(f"{row['number_of_meters']} meters and {row['number_of_years']} year(s) processing time: {round(total_t, 2)} s")
    
    print('Done!')
    return benchmark_df

# Max 3.5 years, because rates exist for that period only. Although theoretically could be extended for + infinite years 
benchmark_df = benchmark(rates_df, number_of_meters=[25, 50, 75, 100, 125, 150, 175, 200], number_of_years=[0.5, 1, 1.5, 2, 2.5, 3, 3.5])

# Plot the benchmarks.

Requires plotly and statsmodels

In [ ]:
import plotly.express as px

fig = px.scatter(benchmark_df, x="number_of_meters", y="execution_time", color="number_of_years", trendline="ols")
fig.show()

fig = px.scatter(benchmark_df, x="number_of_years", y="execution_time", color="number_of_meters", trendline="ols")
fig.show()

# 5.

Function **calc_transportation_cost** appears to scale linearly, though larger datasets should be tested.

A more efficient process of mapping the rates based on the meter data could be implemented. One option would be to reduce the meter data by summing the usage of particular meter for dates resampled into a single rate period.

The process of calculating cost for particular meter could also be parallelised using, for instance, Dask.